In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing
import xgboost as xgb
import scipy as sp
from sklearn.metrics import  make_scorer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression

In [124]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [125]:
print train.shape, test.shape

(891, 12) (418, 11)


In [126]:
Y_train = train.Survived
test_id = test.PassengerId

In [127]:
train_test = pd.concat([train, test])
train_test.head(2)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599


In [128]:
train_test = train_test.drop(['PassengerId', 'Survived', 'Name', 'Cabin'], axis = 1)
print 'shape: '
print train_test.shape, train.shape, test.shape
print '~~~~~~~~~~~~~'
print 'type'
print train_test.dtypes
print '~~~~~~~~~~~~~'
print 'No. of Nans'
print train_test.isnull().sum()
print '~~~~~~~~~~~~~'
train_test.head(2)

shape: 
(1309, 8) (891, 12) (418, 11)
~~~~~~~~~~~~~
type
Age         float64
Embarked     object
Fare        float64
Parch         int64
Pclass        int64
Sex          object
SibSp         int64
Ticket       object
dtype: object
~~~~~~~~~~~~~
No. of Nans
Age         263
Embarked      2
Fare          1
Parch         0
Pclass        0
Sex           0
SibSp         0
Ticket        0
dtype: int64
~~~~~~~~~~~~~


,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,Ticket
0,22.0,S,7.2500,0,3,male,1,A/5 21171
1,38.0,C,71.2833,0,1,female,1,PC 17599


In [129]:
for col in train_test.columns:
    print train_test[col].value_counts()
    print '~~~~~~~~~~~~~~~~~~~~~~~~~'
    print 

24.00    47
22.00    43
21.00    41
30.00    40
18.00    39
25.00    34
28.00    32
36.00    31
26.00    30
29.00    30
27.00    30
19.00    29
23.00    26
32.00    24
20.00    23
35.00    23
31.00    23
45.00    21
33.00    21
17.00    20
39.00    20
16.00    19
42.00    18
40.00    18
34.00    16
50.00    15
47.00    14
48.00    14
38.00    14
2.00     12
         ..
28.50     3
12.00     3
0.83      3
34.50     2
71.00     2
45.50     2
70.00     2
14.50     2
30.50     2
0.92      2
36.50     2
23.50     1
67.00     1
60.50     1
38.50     1
0.17      1
55.50     1
0.33      1
20.50     1
24.50     1
66.00     1
0.42      1
70.50     1
11.50     1
26.50     1
76.00     1
22.50     1
74.00     1
80.00     1
0.67      1
Name: Age, Length: 98, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~

S    914
C    270
Q    123
Name: Embarked, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~

8.0500     60
13.0000    59
7.7500     55
26.0000    50
7.8958     49
10.5000    35
7.7750     26
7.2292     24
7.9250    

In [130]:
# print 'fill NaNs with mean or mode for object type'
# train['Age'].fillna((train['Age'].mean()), inplace=True)
# train['Fare'].fillna((train['Fare'].mean()), inplace=True)
# train['Embarked'].fillna((train.Embarked.value_counts()[0]), inplace=True)
# test['Age'].fillna((test['Age'].mean()), inplace=True)
# test['Fare'].fillna((test['Fare'].mean()), inplace=True)
# test['Embarked'].fillna((test.Embarked.value_counts()[0]), inplace=True)
# train_test = pd.concat([train, test])
# train_test = train_test.drop(['PassengerId', 'Survived', 'Name', 'Cabin', 'Ticket'], axis = 1)

# print 'No. of Nans'
# print train_test.isnull().sum()
# train_test.head(2)

In [131]:
print 'process object types: ["Embarked", "Ticket", "Sex"]'
train_test['Embarked'] = train_test['Embarked'].factorize()[0]
train_test['Ticket'] = train_test['Ticket'].factorize()[0]
train_test = pd.concat([train_test, pd.get_dummies(train_test.Sex)], axis = 1)
train_test = train_test.drop(['Sex'], axis = 1)
print train_test.dtypes
train_test.head()

process object types: ["Embarked", "Ticket", "Sex"]
Age         float64
Embarked      int64
Fare        float64
Parch         int64
Pclass        int64
SibSp         int64
Ticket        int64
female        uint8
male          uint8
dtype: object


,Age,Embarked,Fare,Parch,Pclass,SibSp,Ticket,female,male
0,22.0,0,7.2500,0,3,1,0,0,1
1,38.0,1,71.2833,0,1,1,1,1,0
2,26.0,0,7.9250,0,3,0,2,1,0
3,35.0,0,53.1000,0,1,1,3,1,0
4,35.0,0,8.0500,0,3,0,4,0,1


In [132]:
train_test.describe()

,Age,Embarked,Fare,Parch,Pclass,SibSp,Ticket,female,male
count,1046.000000,1309.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,29.881138,0.392666,33.295479,0.385027,2.294882,0.498854,405.414057,0.355997,0.644003
std,14.413493,0.655586,51.758668,0.865560,0.837836,1.041658,267.867110,0.478997,0.478997
min,0.170000,-1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,0.000000,7.895800,0.000000,2.000000,0.000000,166.000000,0.000000,0.000000
50%,28.000000,0.000000,14.454200,0.000000,3.000000,0.000000,373.000000,0.000000,1.000000
75%,39.000000,1.000000,31.275000,0.000000,3.000000,1.000000,627.000000,1.000000,1.000000
max,80.000000,2.000000,512.329200,9.000000,3.000000,8.000000,928.000000,1.000000,1.000000


In [133]:
# def normalize(columns):
#     for col in columns:
#         mu = train[col].mean()
#         sigma = train[col].std()
#         train_test[col] = (train[col] - train[col].mean())/train[col].std()

In [134]:
# print 'normalize columns ["Age", "Fare"]: '
# columns_to_normalize = ['Age', 'Fare']
# normalize(columns_to_normalize)
# train_test.describe()

In [135]:
X_train = train_test[:len(train)]
X_test = train_test[len(train):]
print X_train.shape, X_test.shape

(891, 9) (418, 9)


In [136]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'binary:logistic',
    'eval_metric': 'rmse',
    'silent': 0
}

In [137]:
dtrain = xgb.DMatrix(X_train, Y_train)
dtest = xgb.DMatrix(X_test)
xgboost_model = xgb.train(dict(xgb_params), dtrain, num_boost_round=1000)

1 0


In [138]:
print 'predict result by xgboost:'
y_predict_xgb = xgboost_model.predict(dtest)
# y_predict_xgb = [1 if p > 0.5 else 0 for p in y_predict_xgb]
print len(test_id), len(y_predict_lr)
output = pd.DataFrame({'PassengerId': test_id, 'Survived': y_predict_xgb})
output.to_csv('Titanic_xgb.csv', index=False)
output.head()

0    272
1    146
Name: Survived, dtype: int64
Done!
